# Connecting a Jupyter Notebook to an Autonomous Database Security Using OCI Vault Secrets
##### _Author: Chad Russell, September 2021_

### Prerequisites 
 - Create Dynamic Group in the respective OCI Tenancy
 - Create a matching rule that matches a resource such as compartment_id to the group
 - Write an OCI policy assigning the Dynamic Group the needed privileges for the activity

In [ ]:
import oci

In [ ]:
# By default this will hit the auth service in the region returned by http://169.254.169.254/opc/v1/instance/region on the instance.
			
signer = oci.auth.signers.InstancePrincipalsSecurityTokenSigner()
identity_client = oci.identity.IdentityClient(config={}, signer=signer)


In [ ]:
print(signer)

In [ ]:
print(identity_client)

 ### Defining Target Compartment, Initializing Resource Principal Authentication, Returning Token and Retrieving List of Autonomous Databases

In [ ]:
# coding: utf-8
# Copyright (c) 2016, 2021, Oracle and/or its affiliates.  All rights reserved.
# This software is dual-licensed to you under the Universal Permissive License (UPL) 1.0 as shown at https://oss.oracle.com/licenses/upl or Apache License 2.0 as shown at http://www.apache.org/licenses/LICENSE-2.0. You may choose either license.

import oci
import sys

"""
Example showing how to initialize and use the Resource Principals signer.

This example shows the resource principals signer being used with the
DBaaS service.  The instance must be set up for resource principals for
this example to work.

The compartment ID must be provided when running the example.

python resource_principals_example <compartment_id>
"""



compartment_id = 'ocid.*******'

# Create a Response Pricipals signer
print("=" * 80)
print("Intializing new signer")
rps = oci.auth.signers.get_resource_principals_signer()

# Print the Resource Principal Security Token
# This step is not required to use the signer, it just shows that the security
# token can be retrieved from the signer.
print("=" * 80)
print("Resource Principal Security Token")
print(rps.get_security_token())

print("=" * 80)
print("Calling list_autonomous_databases")
# Note that the config is passed in as an empty dictionary.  A populated config
# is not needed when using a Resource Principals signer
db_client = oci.database.DatabaseClient({}, signer=rps)
response = db_client.list_autonomous_databases(compartment_id, limit=5)
print(response.data)

### Initializing Vault Client and Listing Secrets from Compartment

In [ ]:
vaults_client = oci.vault.VaultsClient({}, signer=rps)
vaults_client_composite = oci.vault.VaultsClientCompositeOperations(vaults_client)

response = vaults_client.list_secrets(compartment_id)
print(response.data)

### Create Instance of Vault Client Composite

In [ ]:
vaults_client_composite = oci.vault.VaultsClientCompositeOperations(vaults_client)

In [ ]:
print(vaults_client_composite)

### Read Secret Content

##### For this example, manually copy the OCID of a secret id from the output of the 'Initializing Vault Client and Listing Secrets from Compartment' results cell that you would like to use.

In [ ]:
import base64

In [ ]:
# Usage : python secret_examples.py secret_id
def read_secret_value(secret_client, secret_id):
    print("Reading vaule of secret_id {}.".format(secret_id))
     
    response = secret_client.get_secret_bundle(secret_id)
     
    base64_Secret_content = response.data.secret_bundle_content.content
    base64_secret_bytes = base64_Secret_content.encode('ascii')
    base64_message_bytes = base64.b64decode(base64_secret_bytes)
    secret_content = base64_message_bytes.decode('ascii')
     
    return secret_content
     
#Paste the OCID of the secret from the 'Initializing Vault Client and Listing Secrets from Compartment' results cell that you would like to use
 
secret_id = 'ocid1.****'
 
secret_client = oci.secrets.SecretsClient({}, signer=rps)
secret_content = read_secret_value(secret_client, secret_id)
print("Decoded content of the secret is: {}.".format(secret_content))

###  Now go into OCI Console and change value of secret and re-run the commands above to see different secret value

In [ ]:
# Usage : python secret_examples.py secret_id
def read_secret_value(secret_client, secret_id):
    print("Reading vaule of secret_id {}.".format(secret_id))
     
    response = secret_client.get_secret_bundle(secret_id)
     
    base64_Secret_content = response.data.secret_bundle_content.content
    base64_secret_bytes = base64_Secret_content.encode('ascii')
    base64_message_bytes = base64.b64decode(base64_secret_bytes)
    secret_content = base64_message_bytes.decode('ascii')
     
    return secret_content
     
#Paste the OCID of the secret from the 'Initializing Vault Client and Listing Secrets from Compartment' results cell that you would like to use
 
secret_id = 'ocid1.***'
 
secret_client = oci.secrets.SecretsClient({}, signer=rps)
secret_content = read_secret_value(secret_client, secret_id)
print("Decoded content of the secret is: {}.".format(secret_content))

### Now let's work on storing the wallet as secret information

#### Next create each file as a secret in OCI Vault
    - Use the following command to copy the base64 info to your clipboard from a terminal window (if you are using a Mac).  
      Otherwise if you are copying make sure you do not include line or space characters. 

In [ ]:
pwd

In [ ]:
cd /home/datascience/instantclient_21_3/network/admin/base64-wallet


In [ ]:
cat cwallet.sso

In [ ]:
ls

#### Rough Test of Oracle Database Connection
    - It's a good idea to test db connectivity via a sqlplus terminal to ensure proper network connectivity first

In [ ]:
import os
import cx_Oracle

In [ ]:
os.environ['TNS_ADMIN'] = "/home/datascience/instantclient_21_3/network/admin"

In [ ]:

pword = '***********'
p_username = 'admin'
p_password = '*************'
p_service = 'db202108311946_high'
con = cx_Oracle.connect(p_username, p_password, p_service)

print(con)
print(con.version)


In [ ]:
cur = con.cursor()
cur.execute("SELECT table_name FROM dba_tables")
res = cur.fetchall()
for row in res:
    print(row)

### Time to extract the wallet secrets content and password to build the wallet contents
 

#### Step 1 - List the secrets and grab the relevant ones (map to wallet files and db password)

In [ ]:
vaults_client = oci.vault.VaultsClient({}, signer=rps)
vaults_client_composite = oci.vault.VaultsClientCompositeOperations(vaults_client)

response = vaults_client.list_secrets(compartment_id)
print(response.data)

In [ ]:
cwallet = 'ocid1.***'

In [ ]:
# Usage : python secret_examples.py secret_id
# cwallet.sso extraction
def read_secret_value(secret_client, secret_id):
    print("Reading vaule of secret_id {}.".format(secret_id))
     
    response = secret_client.get_secret_bundle(secret_id)
     
    base64_Secret_content = response.data.secret_bundle_content.content
    base64_message_bytes = base64.b64decode(base64_Secret_content)
    
     
    return base64_message_bytes
     
#Paste the OCID of the secret from the respective secret results cell that you would like to use

#This is extracting the cwallet secret contents, base64 decoding and and writing bytes to file names cwallet.sso in file system
secret_id = 'ocid1.***'
 
secret_client = oci.secrets.SecretsClient({}, signer=rps)
secret_content = read_secret_value(secret_client, secret_id)
print("Decoded content of the secret is: {}.".format(secret_content))

f = open("/home/datascience/instantclient_21_3/network/admin/decodedwallet/cwallet.sso", "wb")
f.write(secret_content)
f.close()

In [ ]:
truststore = 'ocid1.***'

In [ ]:
# Read truststore secret from OCI vault.   Base64 decode back into binary and write to file in wallet folder. 
def read_secret_value(secret_client, secret_id):
    print("Reading vaule of secret_id {}.".format(secret_id))
     
    response = secret_client.get_secret_bundle(secret_id)
     
    base64_Secret_content = response.data.secret_bundle_content.content
    base64_message_bytes = base64.b64decode(base64_Secret_content)
    
     
    return base64_message_bytes
     
#Paste the OCID of the secret from the respective secret results cell that you would like to use

#This is extracting the cwallet secret contents, base64 decoding and and writing bytes to file names cwallet.sso in file system
secret_id = 'ocid1.***'
 
secret_client = oci.secrets.SecretsClient({}, signer=rps)
secret_content = read_secret_value(secret_client, secret_id)
print("Decoded content of the secret is: {}.".format(secret_content))

f = open("/home/datascience/instantclient_21_3/network/admin/decodedwallet/truststore.jks", "wb")
f.write(secret_content)
f.close()


In [ ]:
tnsnames = 'ocid1.***'

In [ ]:
# tnsnames extraction from OCI vault secret and write to file system
def read_secret_value(secret_client, secret_id):
    print("Reading vaule of secret_id {}.".format(secret_id))
     
    response = secret_client.get_secret_bundle(secret_id)
     
    base64_Secret_content = response.data.secret_bundle_content.content
    base64_secret_bytes = base64_Secret_content.encode('ascii')
    base64_message_bytes = base64.b64decode(base64_secret_bytes)
    secret_content = base64_message_bytes.decode('ascii')
     
    return secret_content
     
#Paste the OCID of the secret from the 'Initializing Vault Client and Listing Secrets from Compartment' results cell that you would like to use
 
secret_id = 'ocid.***'
 
secret_client = oci.secrets.SecretsClient({}, signer=rps)
secret_content = read_secret_value(secret_client, secret_id)
print("Decoded content of the secret is: {}.".format(secret_content))


f = open("/home/datascience/instantclient_21_3/network/admin/decodedwallet/tnsnames.ora", "w")
f.write(secret_content)
f.close()

In [ ]:
sqlnet = 'ocid1.***'

In [ ]:
# sqlnet.ora extraction from OCI vault secret and write to file system
def read_secret_value(secret_client, secret_id):
    print("Reading vaule of secret_id {}.".format(secret_id))
     
    response = secret_client.get_secret_bundle(secret_id)
     
    base64_Secret_content = response.data.secret_bundle_content.content
    base64_secret_bytes = base64_Secret_content.encode('ascii')
    base64_message_bytes = base64.b64decode(base64_secret_bytes)
    secret_content = base64_message_bytes.decode('ascii')
     
    return secret_content
     
#Paste the OCID of the secret from the 'Initializing Vault Client and Listing Secrets from Compartment' results cell that you would like to use
 
secret_id = 'ocid1.***'
 
secret_client = oci.secrets.SecretsClient({}, signer=rps)
secret_content = read_secret_value(secret_client, secret_id)
print("Decoded content of the secret is: {}.".format(secret_content))


f = open("/home/datascience/instantclient_21_3/network/admin/decodedwallet/sqlnet.ora", "w")
f.write(secret_content)
f.close()

In [ ]:
ojdbc = 'ocid1.***'

In [ ]:
# ojdbc.properties file retrieval, base64 decoding and writing to wallet folder
def read_secret_value(secret_client, secret_id):
    print("Reading vaule of secret_id {}.".format(secret_id))
     
    response = secret_client.get_secret_bundle(secret_id)
     
    base64_Secret_content = response.data.secret_bundle_content.content
    base64_secret_bytes = base64_Secret_content.encode('ascii')
    base64_message_bytes = base64.b64decode(base64_secret_bytes)
    secret_content = base64_message_bytes.decode('ascii')
     
    return secret_content
     
#Paste the OCID of the secret from the 'Initializing Vault Client and Listing Secrets from Compartment' results cell that you would like to use
 
secret_id = 'ocid1.***'
 
secret_client = oci.secrets.SecretsClient({}, signer=rps)
secret_content = read_secret_value(secret_client, secret_id)
print("Decoded content of the secret is: {}.".format(secret_content))


f = open("/home/datascience/instantclient_21_3/network/admin/decodedwallet/ojdbc.properties", "w")
f.write(secret_content)
f.close()

In [ ]:
keystore = 'ocid1.***'

In [ ]:
# Read keystore secret from OCI vault.   Base64 decode back into binary and write to file in wallet folder. 
def read_secret_value(secret_client, secret_id):
    print("Reading vaule of secret_id {}.".format(secret_id))
     
    response = secret_client.get_secret_bundle(secret_id)
     
    base64_Secret_content = response.data.secret_bundle_content.content
    base64_message_bytes = base64.b64decode(base64_Secret_content)
    
     
    return base64_message_bytes
     
#Paste the OCID of the secret from the respective secret results cell that you would like to use

#This is extracting the cwallet secret contents, base64 decoding and and writing bytes to file names keystore.jks in file system
secret_id = 'ocid1.***'
 
secret_client = oci.secrets.SecretsClient({}, signer=rps)
secret_content = read_secret_value(secret_client, secret_id)
print("Decoded content of the secret is: {}.".format(secret_content))

f = open("/home/datascience/instantclient_21_3/network/admin/decodedwallet/keystore.jks", "wb")
f.write(secret_content)
f.close()

In [ ]:
ewallet = 'ocid1.***'

In [ ]:
# Read ewallet secret from OCI vault.   Base64 decode back into binary and write to file in wallet folder. 
def read_secret_value(secret_client, secret_id):
    print("Reading vaule of secret_id {}.".format(secret_id))
     
    response = secret_client.get_secret_bundle(secret_id)
     
    base64_Secret_content = response.data.secret_bundle_content.content
    base64_message_bytes = base64.b64decode(base64_Secret_content)
    
     
    return base64_message_bytes
     
#Paste the OCID of the secret from the respective secret results cell that you would like to use

#This is extracting the cwallet secret contents, base64 decoding and and writing bytes to file names ewallet.p12 in file system
secret_id = 'ocid1.***'
 
secret_client = oci.secrets.SecretsClient({}, signer=rps)
secret_content = read_secret_value(secret_client, secret_id)
print("Decoded content of the secret is: {}.".format(secret_content))

f = open("/home/datascience/instantclient_21_3/network/admin/decodedwallet/ewallet.p12", "wb")
f.write(secret_content)
f.close()

In [ ]:
dbpw = 'ocid1.***'

In [ ]:
# database password secret extraction
def read_secret_value(secret_client, secret_id):
    print("Reading value of secret_id {}.".format(secret_id))
     
    response = secret_client.get_secret_bundle(secret_id)
     
    base64_Secret_content = response.data.secret_bundle_content.content
    base64_secret_bytes = base64_Secret_content.encode('ascii')
    base64_message_bytes = base64.b64decode(base64_secret_bytes)
    secret_content = base64_message_bytes.decode('ascii')
     
    return secret_content
     
#Paste the OCID of the secret from the 'Initializing Vault Client and Listing Secrets from Compartment' results cell that you would like to use
 
secret_id = 'ocid1.***'
 
secret_client = oci.secrets.SecretsClient({}, signer=rps)
secret_content = read_secret_value(secret_client, secret_id)
print("Decoded content of the secret is: {}.".format(secret_content))



#### Now we will use the newly constructed wallet and retrieved password from OCI Vault to securely initiate connectivity with Oracle's Autonomous JSON Database!

In [ ]:
os.environ['TNS_ADMIN'] = '/home/datascience/instantclient_21_3/network/admin/decodedwallet/'

In [ ]:

pword = secret_content
p_username = 'admin'
p_password = secret_content
p_service = 'db202108311946_high'
con4 = cx_Oracle.connect(p_username, p_password, p_service)

print(con4)
print(con4.version)

In [ ]:
cur4 = con4.cursor()
cur4.execute("SELECT table_name FROM dba_tables")
res4 = cur4.fetchall()
for row in res4:
    print(row)